In [0]:
#pip install --upgrade tensorflow-gpu
import tensorflow as tf
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
import numpy as np
import tensorflow_datasets as tfds
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, Lambda, Dense
from tensorflow.keras.models import Model
from sklearn import preprocessing
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv1D, Reshape, Concatenate

In [0]:
data = pd.read_csv('QDeveloper.csv')
data = data.sample(frac=1)
data = data.fillna('unknown')
y = list(data['FixedByID']) 
x_context = list(data['Title_Description'])
data.AST = data.AST.astype(str)
x_AST = list(data['AST'])

le = preprocessing.LabelEncoder()
le.fit(y)

categories = list(set(y))

def encode(le, labels):
    enc = le.transform(labels)
    #return keras.utils.to_categorical(enc)
    return enc

def decode(le, labels):
    #dec = np.argmax(one_hot, axis=1)
    #dec = np.argmax(one_hot)
    dec = le.inverse_transform(labels)
    return dec

test = encode(le,categories)
untest = decode(le, test)


y_enc = encode(le, y)

#80% / 20% train / test split:

train_size = int(len(x_context) * .8)

#x_train = np.asarray(x_enc[:train_size])
#y_train = np.asarray(y_enc[:train_size])

#x_test = np.asarray(x_enc[train_size:])
#y_test = np.asarray(y_enc[train_size:]) 

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

x_train_context = x_context[:train_size]
x_train_AST = x_AST[:train_size]
y_train = y_enc[:train_size]

x_test_context = x_context[train_size:]
x_test_AST = x_AST[train_size:]
y_test = y_enc[train_size:]

# convert string to lower case 
x_train_context = [s.lower() for s in x_train_context]
x_test_context = [s.lower() for s in x_test_context]
#=======================Convert string to index================
# Tokenizer
tk_context = Tokenizer(num_words=None, char_level=None, oov_token='Unknown')
tk_context.fit_on_texts(x_context)

tk_AST = Tokenizer(num_words=None, char_level=None, oov_token='Unknown')
tk_AST.fit_on_texts(x_AST)

# Convert string to index 
x_train_context_sequences = tk_context.texts_to_sequences(x_train_context)
x_train_AST_sequences = tk_context.texts_to_sequences(x_train_AST)
x_test_context_sequences = tk_AST.texts_to_sequences(x_test_context)
x_test_AST_sequences = tk_AST.texts_to_sequences(x_test_AST)

# Padding
x_train_context = pad_sequences(x_train_context_sequences, maxlen=1000, padding='post')
x_train_AST = pad_sequences(x_train_AST_sequences, maxlen=1000, padding='post')
x_test_context = pad_sequences(x_test_context_sequences, maxlen=1000, padding='post')
x_test_AST = pad_sequences(x_test_AST_sequences, maxlen=1000, padding='post')

# Convert to numpy array
x_train_context = np.array(x_train_context)
x_train_AST = np.array(x_train_AST)
x_test_context = np.array(x_test_context)
x_test_AST = np.array(x_test_AST)

#y_train_format = tf.shape(tf.squeeze(y_train[0]))
#y_test_format = tf.shape(tf.squeeze(y_test[0]))

trainingcontextdataset = (
    tf.data.Dataset.from_tensor_slices(
        (
             tf.cast(x_train_context, tf.float32),
             tf.cast(y_train, tf.float32)          
        )
    )
)

trainingASTdataset = (
    tf.data.Dataset.from_tensor_slices(
        (
             tf.cast(x_train_AST, tf.float32),
             tf.cast(y_train, tf.float32)          
        )
    )
)

testcontextdataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(x_test_context, tf.float16),
            tf.cast(y_test, tf.float32)           
        )
    )
)

testASTdataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(x_test_AST, tf.float16),
            tf.cast(y_test, tf.float32)           
        )
    )
)

In [0]:
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates
  
def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)
  
  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
  
  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    
  pos_encoding = angle_rads[np.newaxis, ...]
    
  return tf.cast(pos_encoding, dtype=tf.float32)

def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
  ])

def scaled_dot_product_attention(q, k, v, mask):
  """Calculate the attention weights.
  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  The mask has different shapes depending on its type(padding or look ahead) 
  but it must be broadcastable for addition.
  
  Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable 
          to (..., seq_len_q, seq_len_k). Defaults to None.
    
  Returns:
    output, attention_weights
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)
  
  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # add the mask to the scaled tensor.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)  

  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return output, attention_weights

def create_padding_mask(seq):
  seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
  
  # add extra dimensions to add the padding
  # to the attention logits.
  return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)
  
def create_masks(inp):
    # Encoder padding mask
    enc_padding_mask = create_padding_mask(inp)

    return enc_padding_mask

def TransformerEncoder(x, contextflag):   
   if contextflag == True:
     ##################Initalize Context Variable###################
     input_vocab_size = len(tk_context.word_index) + 2
     target_vocab_size = len(tk_context.word_index) + 2
     enc_padding_mask = create_masks(x)
     tranformer = Transformer(num_layers, d_model, num_heads, dff,
                          len(tk_context.word_index) + 1, len(tk_context.word_index) + 1, dropout_rate)
     t_out = tranformer(x, True,enc_padding_mask)
   else:
     input_vocab_size = len(tk_AST.word_index) + 2
     target_vocab_size = len(tk_AST.word_index) + 2
     enc_padding_mask = create_masks(x)
     tranformer = Transformer(num_layers, d_model, num_heads, dff,
                          len(tk_context.word_index) + 1, len(tk_context.word_index) + 1, dropout_rate)
     t_out = tranformer(x, True,enc_padding_mask)

   return t_out

In [0]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model
    
    assert d_model % self.num_heads == 0
    
    self.depth = d_model // self.num_heads
    
    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)
    
    self.dense = tf.keras.layers.Dense(d_model)
        
  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])
    
  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]
    
    q = self.wq(q)  # (batch_size, seq_len, d_model)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
    v = self.wv(v)  # (batch_size, seq_len, d_model)
    
    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)
    
    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)
    
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention, 
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)
        
    return output, attention_weights

class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    
    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    
  def call(self, x, training, mask):

    attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
    attn_output = self.dropout1(attn_output, training=training)
    out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)
    
    ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
    ffn_output = self.dropout2(ffn_output, training=training)
    out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)    
    return out2

class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
               maximum_position_encoding, rate=0.1):
    super(Encoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers
    
    self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding, 
                                            self.d_model)   
    
    self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) 
                       for _ in range(num_layers)]
  
    self.dropout = tf.keras.layers.Dropout(rate)
        
  def call(self, x, training, mask):

    seq_len = tf.shape(x)[1]

    # adding embedding and position encoding.
    x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)
    
    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask)
    
    return x  # (batch_size, input_seq_len, d_model)

class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, 
                   target_vocab_size, rate=0.1):
        super(Transformer, self).__init__()

        self.encoder = Encoder(num_layers, d_model, num_heads, dff, 
                               input_vocab_size, rate)

        self.dense1 = tf.keras.layers.Dense(d_model, activation='tanh')
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dense2 = tf.keras.layers.Dense(256, activation='tanh')
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.final_layer = tf.keras.layers.Dense(len(categories), activation='softmax')
        #self.final_layer = tf.keras.layers.Dense(128, activation='relu')

    def call(self, inp, training, enc_padding_mask):
        enc_output = self.encoder(inp, training, enc_padding_mask)
        enc_output = self.dense1(enc_output[:,0])
        enc_output = self.dropout1(enc_output, training=training)
        enc_output = self.dense2(enc_output)
        enc_output = self.dropout2(enc_output, training=training)
        final_output = self.final_layer(enc_output)
        return final_output
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [0]:
###########################Initalize Variable#####################################
num_layers = 6
d_model = 512
dff = 2048
num_heads = 8

dropout_rate = 0.1
learning_rate = CustomSchedule(d_model)
optimizer = tf.keras.optimizers.Adam(learning_rate)

In [0]:
TransformerEncoder(x_train_context[22:23],True)

In [0]:
TransformerEncoder(x_train_AST[5:6], False)

In [0]:
tf.keras.layers.Concatenate()([TransformerEncoder(x_train_context[22:23],True), TransformerEncoder(x_train_AST[5:6], False)]).shape

In [0]:
num_layers = 6
d_model = 512
dff = 2048
num_heads = 8
dropout_rate = 0.1
enc_padding_mask = create_masks(x_train_AST[5:6])
input_vocab_size = len(tk_AST.word_index) + 2
target_vocab_size = len(tk_AST.word_index) + 2
embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
x = embedding(enc_padding_mask)  # (batch_size, input_seq_len, d_model)
x *= tf.math.sqrt(tf.cast(d_model, tf.float32))
x

In [10]:
def codeembedding(x): 
    embedsize = len(tk_context.word_index) + 1
    result=  tf.keras.layers.Embedding(embedsize, output_dim=d_model)(x)
    return result


input_context = Input(shape=(1000,), dtype=tf.float32)
input_AST = Input(shape=(1000,), dtype=tf.float32)

contextmodel = Model(inputs=input_context, outputs=TransformerEncoder(input_context,True))
#codemodel = Model(inputs=input_AST, outputs=TransformerEncoder(input_AST, False))

codedense = tf.keras.layers.Dense(256, activation='relu')(codeembedding(input_AST))
LSTM = tf.keras.layers.LSTM(128)(codedense)
outputs = tf.keras.layers.Dense(len(categories), activation='softmax')(LSTM)
codemodel = Model(inputs=input_AST, outputs=outputs)

# the Context and AST
combinedInput = tf.keras.layers.Concatenate()([contextmodel.output, codemodel.output])

#final_Output Softmax
#final_output = Dense(536, activation="relu")(codemodel)
final_output = Dense(258, activation="tanh")(combinedInput)
final_output = Dense(len(categories), activation="softmax")(combinedInput)

model = Model(inputs=[contextmodel.input, codemodel.input], outputs=final_output)

model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit([x_train_context, x_train_AST], y_train, epochs=5)
model.evaluate([x_test_context, x_test_AST],y_test)
model.save_weights('./model_PreditDeveloper.h5')

Epoch 1/5
42/42 [==============================] - 2241s 53s/step - loss: 5.5898 - accuracy: 0.0061
Epoch 2/5
42/42 [==============================] - 2221s 53s/step - loss: 5.5847 - accuracy: 0.0061
Epoch 3/5
42/42 [==============================] - 2228s 53s/step - loss: 5.5766 - accuracy: 0.0122
Epoch 4/5
42/42 [==============================] - 2305s 55s/step - loss: 5.5741 - accuracy: 0.0122
Epoch 5/5
11/11 [==============================] - 202s 18s/step - loss: 5.5753 - accuracy: 0.0091


[5.575272083282471, 0.00909090880304575]

In [0]:
model.save_weights('./model_PredictDeveloper.h5')

In [0]:
model.load_weights('./model_PredictDeveloper.h5')  
predicts = model.predict(x_test, batch_size=32)  

In [0]:
y_test = np.asarray(y_enc[train_size:])
y_test = decode(le, y_test)
y_preds = decode(le, predicts)

from sklearn import metrics

metrics.confusion_matrix(y_test, y_preds)

print(metrics.classification_report(y_test, y_preds))

In [127]:
model.summary()

Model: "model_71"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_65 (InputLayer)           [(None, 1000)]       0                                            
__________________________________________________________________________________________________
input_66 (InputLayer)           [(None, 1000)]       0                                            
__________________________________________________________________________________________________
tf_op_layer_Equal_32 (TensorFlo [(None, 1000)]       0           input_65[0][0]                   
__________________________________________________________________________________________________
embedding_118 (Embedding)       (None, 1000, 512)    3192320     input_66[0][0]                   
___________________________________________________________________________________________